In [1]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
from shapely.geometry import Point

In [2]:
pwd

'/home/vipul/v6/dbupdate'

In [4]:
df_raw_tracks = pd.read_excel('./dist_1426_1436.xlsx', engine='openpyxl')

df_tracks = pd.DataFrame()
df_tracks['unique_ID'] = df_raw_tracks['track_id']

df_raw_tracks['intersect_point1'] = df_raw_tracks['intersect_point1'].apply(lambda s: Point(*map(float, s.strip('POINT ()').split())))
df_raw_tracks['intersect_point2'] = df_raw_tracks['intersect_point2'].apply(lambda s: Point(*map(float, s.strip('POINT ()').split())))

df_tracks['start_x'] = df_raw_tracks['intersect_point1'].apply(lambda point: point.x)
df_tracks['start_y'] = df_raw_tracks['intersect_point1'].apply(lambda point: point.y)
df_tracks['end_x'] = df_raw_tracks['intersect_point2'].apply(lambda point: point.x)
df_tracks['end_y'] = df_raw_tracks['intersect_point1'].apply(lambda point: point.y)

df_tracks['enter_frame'] = df_raw_tracks['frame_num1']
df_tracks['exit_frame'] = df_raw_tracks['frame_num2']

df_tracks['enter_time'] = df_raw_tracks['time1']
df_tracks['exit_time'] = df_raw_tracks['time2']

df_tracks['frame_diff'] = df_raw_tracks['frame_diff']
df_tracks['elapsed_time'] = df_raw_tracks['elapsed_time']
df_tracks['speed'] = df_raw_tracks['speed']
df_tracks['lane'] = df_raw_tracks['lane']
df_tracks


,unique_ID,start_x,start_y,end_x,end_y,enter_frame,exit_frame,enter_time,exit_time,frame_diff,elapsed_time,speed,lane
0,142623,823.577175,941.278765,1276.926869,941.278765,128,142,2023-07-21 14:26:55.233,2023-07-21 14:26:55.700,14,0.466667,19.165714,lane1
1,142624,567.761777,863.832275,1003.014556,863.832275,101,112,2023-07-21 14:26:54.333,2023-07-21 14:26:54.700,11,0.366667,24.392727,lane3
2,142632,604.391748,871.563751,1071.953328,871.563751,20,32,2023-07-21 14:26:51.633,2023-07-21 14:26:52.033,12,0.400000,22.360000,lane2
3,142634,810.016081,935.351400,1260.364893,935.351400,26,41,2023-07-21 14:26:51.833,2023-07-21 14:26:52.333,15,0.500000,17.888000,lane1
4,142647,521.103707,844.774754,986.810409,844.774754,70,78,2023-07-21 14:26:53.300,2023-07-21 14:26:53.567,8,0.266667,33.540000,lane3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,14351962,520.486139,844.522508,981.114224,844.522508,5121,5129,2023-07-21 14:35:48.667,2023-07-21 14:35:48.933,8,0.266667,33.540000,lane3
696,14351964,803.289781,932.411427,1251.610356,932.411427,5127,5138,2023-07-21 14:35:48.867,2023-07-21 14:35:49.233,11,0.366667,24.392727,lane1
697,14351980,503.765751,837.693053,969.163522,837.693053,5182,5191,2023-07-21 14:35:50.700,2023-07-21 14:35:51.000,9,0.300000,29.813333,lane3
698,14351986,525.968539,846.761798,970.578616,846.761798,5208,5215,2023-07-21 14:35:51.567,2023-07-21 14:35:51.800,7,0.233333,38.331429,lane3


In [5]:
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='computerserver',
    database='radishdb'
)


In [6]:

# Convert DataFrame to a list of tuples
records = list(df_tracks.itertuples(index=False, name=None))

# Create the SQL query template
table_name = 'ProcessedTracks'
columns = ', '.join(df_tracks.columns)
placeholders = ', '.join(['%s'] * len(df_tracks.columns))
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"



In [7]:
insert_query

'INSERT INTO ProcessedTracks (unique_ID, start_x, start_y, end_x, end_y, enter_frame, exit_frame, enter_time, exit_time, frame_diff, elapsed_time, speed, lane) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [8]:
# Execute the query and insert the data
with connection.cursor() as cursor:
    cursor.executemany(insert_query, records)

connection.commit()


connection.close()